# DAGs - a generalization of the alignment problem
##Specification
- Name: problem15 in Rosalind
- Name your notebook as: DAG.ipynb
- options: none
- input: filename passed as first parameter to main
- output: a text file. ( using print ( .... file=someFileObject) is a handy way to do this after you have opened someFileObject as a text file). I find it handy to name these files by creating a string by concatenating the string named infile with ".out" ... rosalind5.txt.out ( for example).
-Rosalind Problem Name: Find the longest path in a DAG

As always, include a Inspection Intro Markdown that describes your specific algorithm at the beginning of the notebook, and another Inspection Results markdown at the end of the notebook that documents: your inspection team, the findings of the team, and your resolution of those findings.

Please submit your notebook, an example of one of the Rosalind files that you ran and passed, and the output that your program generated as a text file.

## Description
As a prelude to an upcoming assignment in profile alignments, please consider the related problem of generalized weighted-DAGs (directed acyclic graph). Rosalind has a very nice example that includes weighted edges connecting nodes. As applied to a simple pairwise alignment, nodes represent a specific assignment of a character pairing between a character in one of the sequences to either an initial insert/delete (indel), an extension of that indel, or a match/mismatch. 

Unlike classical pairwise alignment, the specific order to consider the resolution of scores in the graph is established using the topology of the graph ( topological ordering). This ordering will avoid recursion in developing the final score. This is another way of using Dynamic programming in a more generalized way. ( pp247-52 in Compeau and the detour on pp287-88).

## Hints
1) scoring of nodes that have no inputs (or possibly incomplete inputs) must be done carefully. Choose these wisely. In a pairwise alignment, these nodes would be considered boundary nodes.

2) The problem is quite clear on the starting/ending node to consider. Make sure to evaluate these properly.

3) each node can have many inputs or none. In the generalized DAG, there is no limit on the number of inputs. Consider a data structure here that allows you to represent the incoming edge along with a score that is computed by adding the weight of that incoming edge to the score of its preceding node.

4) As you write these classes, consider that these may be usable in the future.

5) sets and dictionaries require keys that are immutable. Those keys can be integers, or floats, or tuples. Think carefully about using tuples as containers-that-can-be-keys.

6) python has a container called a namedtuple. These are available in the container module. They are quite handy.

7) python has two functions called any() and all(); any() evaluates to true if any of its iterable arguments are True; all() evaluates to True if all of its iterable arguments are true. None, for example, evaluates to False. 

8) consider implementing a helper method called _argMax_. If you happen to have a dictionary of values, it returns the key whose paired-value is maximal among all values in the dictionary.

9) Carefully consider the implementations that are provided in the text. Specifically, the topology ordering pseudocode removes edges from the constructed DAG. This is a bit annoying, considering that we need that DAG for scoring and backtracking. What is the purpose that removing an edge is attempting to perform? Look over the Wikipedia article on "Dataflow" for an idea as to how you might avoid destroying your graph.

10) My implementation has specific objects that are nodes, edges, and the DAG itself. It is implemented in about 120 lines including whitespace.

11) Rosalind will provide test cases where the proscribed beginning and ending node have inputs/outputs that may induce cycles. In these problems, remember that we are considering the DAG that is described between (inclusively) the start and end node so the inputs to start don't matter; the outputs from end don't matter either.

 

Here is a template to use.

## Inspection Intro

In [1]:
class DAG:
    """
    A class representing a Directed Acyclic Graph (DAG).
    """

    def __init__(self):
        """
        Initializes a new instance of the DAG class.
        """
        self.edges = {}

    def add_node(self, node):
        """
        Adds a node to the graph if it does not already exist.

        :param node: The node to add to the graph.
        """
        if node not in self.edges:
            self.edges[node] = []

    def add_edge(self, from_node, to_node, weight):
        """
        Adds an edge from one node to another with the specified weight.

        :param from_node: The node where the edge starts.
        :param to_node: The node where the edge ends.
        :param weight: The weight of the edge.
        """
        self.add_node(from_node)
        self.add_node(to_node)
        self.edges[from_node].append((to_node, weight))

    def longest_path(self, start, end):
        """
        Finds the longest path from the start node to the end node.

        :param start: The node to start the path.
        :param end: The node to end the path.
        :return: A tuple containing the length of the longest path and the nodes in that path.
        """
        # Initialize the distance to all nodes to be -infinity, except the start node which is 0
        distance = {node: float('-inf') for node in self.edges}
        distance[start] = 0
        # Bookkeeping for path reconstruction
        path = {}

        # Topological sort of the nodes
        topo_order = self.topological_sort()

        # Relax edges in topological order
        for node in topo_order:
            for neighbor, weight in self.edges[node]:
                if distance[neighbor] < distance[node] + weight:
                    distance[neighbor] = distance[node] + weight
                    path[neighbor] = node

        # Reconstruct the longest path
        longest_path = []
        current = end
        while current != start:
            longest_path.append(current)
            current = path.get(current, None)
            if current is None:
                return (None, [])
        longest_path.append(start)
        longest_path.reverse()

        return (distance[end], longest_path)

    def topological_sort(self):
        """
        Performs a topological sort on the graph.

        :return: A list of nodes in topologically sorted order.
        """
        visited = set()
        stack = []

        def recursive_visit(node):
            if node not in visited:
                visited.add(node)
                for neighbor, _ in self.edges.get(node, []):
                    recursive_visit(neighbor)
                stack.append(node)

        for node in self.edges:
            recursive_visit(node)

        return stack[::-1]

def main(inFile=None):
    """
    Reads a graph from a file, computes the longest path between two nodes, and prints the result.

    :param inFile: The path to the file containing the graph data.
    """
    thisDAG = DAG()

    with open(inFile) as fh:
        begin = int(fh.readline().rstrip())
        end = int(fh.readline().rstrip())

        for line in fh:
            parts = line.strip().split('->')
            node = int(parts[0])
            neighbors = parts[1].split(',')
            for neighbor in neighbors:
                n, weight = neighbor.split(':')
                thisDAG.add_edge(node, int(n), int(weight))
    with open("rosalind_ba5d_out.txt", 'w') as file:
        # Find the longest path in the graph
        length, longest_path = thisDAG.longest_path(begin, end)
        if length is not None and longest_path:
            # Print the length and the path
            print(length)
            file.write(str(length)+'\n')
            file.write('->'.join(map(str, longest_path)))
        

        else:
            file.write('No path found')

if __name__ == "__main__":
    main(inFile='rosalind_ba5d.txt')



0


## Inspection Results